# Face Recognition Using LBPH Face Recognizer
# 
## Recognise Face and Launch Kubernetes Deployment

### STEPS :-
1. Create Data Set
2. Train Model
3. Recognise Face

## STEP 1 - Prepare Training Data

In [ ]:
import cv2
import numpy as np

# Load Haarcascade face detector
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Function to crop detected face
def face_extractor(img):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray,1.3,5)
    
    if faces == ():
        return None
    
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h,x:x+w]
    return cropped_face
    
# Initialize Webcam 
cam = cv2.VideoCapture(0)

# Collect 200 samples of face from webcam
count = 0 
while True:
    ret, frame = cam.read()
    
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extracter(frame),(200,200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        
        # Save file in specified directory with each having unique name
        file_name = './Aryanshu/'+str(count)+'.jpg'
        cv2.imwrite(file_name,face)
        
        # Display saved images with count
        cv2.putText(face,str(count),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,[255,255,255],2)
        cv2.imshow('Face Extractor',face)
    
    if cv2.waitKey(1) == 13 or count == 200:
        break
        
cam.release()
cv2.destroyAllWindows()
print("Training Data Collection Complete")

## STEP 2 - Train the Model

In [1]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile , join

# Get the training data we previously made
data_path = './Aryanshu/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path,f))]

# Create arrays for training data and labels
Training_Data, Labels = [],[]

# Open taining images in our data_path
# And, create a numpy array for training data
for i,files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)
    
# Create a numpy array for both training data and labels
Labels = np.asarray(Labels,dtype= np.int32)

# Initialize facial recognizer
model_aryanshu = cv2.face_LBPHFaceRecognizer.create()

# Train the model
model_aryanshu.train(np.asarray(Training_Data),np.asarray(Labels))

print("Model Trained Successfully")

Model Trained Successfully


## STEP 3 - Run Facial Recognition

In [2]:
import cv2
import numpy as np
from os import system
import os

# Load face detector
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi

# Open Webcam
cap = cv2.VideoCapture(0)

# Deployment Launched or Not
launched = False

while True:
    ret, frame = cap.read()
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        results = model_aryanshu.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confidence Score'
        
        # Display Confidence Score
        cv2.putText(image, display_string, (10, 30), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,0), 1)
        
        if confidence > 80:
            cv2.putText(image, "Hello Aryanshu", (350, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 1)
            # Create a K8s deployment when the face is recognised
            system(" kubectl cluster-info > k8s_status.txt")
            
            file = open("k8s_status.txt", "r")
            k8s_status = str(file.read())
            file.close()
            
            if 'running' in k8s_status:
                if launched == False:
                    cv2.putText(image, "Launching Deployment", (1, 400), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0),1)
                    system("kubectl create deployment recognised --image=httpd")
                    launched = True
                else:
                    cv2.putText(image, "Launched Deployment", (1, 400), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0),1)
                    launched = True
            cv2.imshow('Face Recognition', image )
        else:
            cv2.putText(image, "Unknown User", (350, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )
    except:
        cv2.putText(image, "No Face Found", (10, 30) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()

if os.path.exists("k8s_status.txt"):
    os.remove("k8s_status.txt")

<>:13: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:13: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\ARYANS~1\AppData\Local\Temp/ipykernel_7692/272728537.py:13: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():
